# Feature Engineering
Creating basic features from IPL data

In [ ]:
import pandas as pd

df = pd.read_csv('ipl_data.csv')

In [ ]:
# feature: total runs per over
if 'over' in df.columns and 'runs' in df.columns:
    runs_per_over = df.groupby('over')['runs'].sum().reset_index()
    runs_per_over.head()

In [ ]:
# binary feature for wicket
if 'is_wicket' in df.columns:
    df['wicket_flag'] = df['is_wicket'].apply(lambda x: 1 if x > 0 else 0)

df.head()